<a href="https://colab.research.google.com/github/piyushupadhyay07/Topic-Modeling-using-LDA/blob/main/Topic_Modeling_using_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df= pd.read_csv('/Consumer_Complaints_Dataset.csv')

In [ ]:
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/06/20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"XXXX, Transunion and XXXX are reporting a Coll...",Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",GA,30135,Servicemember,Consent provided,Web,03/06/20,Closed with explanation,Yes,NaN,3556195
1,01/26/21,Credit card or prepaid card,General-purpose prepaid card,Problem getting a card or closing an account,"Trouble getting, activating, or registering a ...",This is my 4th compliant to CNB regarding my g...,None,CITY NATIONAL BANK,CA,92620,None,Consent provided,Web,02/11/21,Closed with explanation,Yes,NaN,4095886
2,02/18/20,Mortgage,FHA mortgage,Struggling to pay mortgage,None,I have been working on completing a short sale...,None,Ocwen Financial Corporation,TX,78250,None,Consent provided,Web,02/18/20,Closed with explanation,Yes,NaN,3536395
3,01/26/21,"Payday loan, title loan, or personal loan",Personal line of credit,Charged fees or interest you didn't expect,None,This complaint is against : KeyBank / Key Corp...,None,KEYCORP,GA,31808,None,Consent provided,Web,01/26/21,Closed with monetary relief,Yes,NaN,4094588
4,08/17/20,Debt collection,I do not know,Written notification about debt,Didn't receive notice of right to dispute,This is in reference to closed case XXXX XXXX ...,None,"Debt Recovery Solutions, LLC",AL,35212,None,Consent provided,Web,08/17/20,Closed with explanation,Yes,NaN,3800061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23181,04/25/21,Mortgage,Conventional home mortgage,Trouble during payment process,None,I received a letter dated XX/XX/2021 identifyi...,None,NATIONSTAR MORTGAGE,CA,92026,None,Consent provided,Web,04/25/21,Closed with explanation,Yes,NaN,4326082
23182,12/20/20,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,Solomon and Solomon Pc is attempting to collec...,None,"Solomon and Solomon, P.C.",NY,12751,None,Consent provided,Web,12/20/20,Closed with explanation,No,NaN,4022551
23183,05/08/21,"Money transfer, virtual currency, or money ser...",Foreign currency exchange,Fraud or scam,None,XXXX BAM Trading Services Inc. d.b.a. Binance....,None,BAM Management US Holdings Inc.,AZ,85006,None,Consent provided,Web,05/12/21,Closed with explanation,Yes,NaN,4361861
23184,05/14/21,Credit card or prepaid card,Government benefit card,Problem with a purchase or transfer,Charged for a purchase or transfer you did not...,To Whom it may concern received a certified le...,None,Conduent Incorporated,NY,10553,None,Consent provided,Web,05/14/21,Closed with explanation,Yes,NaN,4376371


In [ ]:
df['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    8250
Mortgage                                                                        4176
Debt collection                                                                 4048
Credit card or prepaid card                                                     3069
Checking or savings account                                                     1372
Vehicle loan or lease                                                            714
Money transfer, virtual currency, or money service                               617
Student loan                                                                     511
Payday loan, title loan, or personal loan                                        429
Name: Product, dtype: int64

In [ ]:
df['Company'].value_counts()

EQUIFAX, INC.                                  2544
TRANSUNION INTERMEDIATE HOLDINGS, INC.         2060
Experian Information Solutions Inc.            1973
CITIBANK, N.A.                                  817
JPMORGAN CHASE & CO.                            735
                                               ... 
Law Offices of Robert A. Schuerger Co., LPA       1
Sunlight Financial LLC                            1
BANKERS HEALTHCARE GROUP LLC                      1
Clark County Collection Service                   1
LoanUnited.com, LLC                               1
Name: Company, Length: 1452, dtype: int64

In [ ]:
complaints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'Complaints'})
complaints_df

,Complaints,Product,Company
0,"XXXX, Transunion and XXXX are reporting a Coll...","Credit reporting, credit repair services, or o...","TRANSUNION INTERMEDIATE HOLDINGS, INC."
1,This is my 4th compliant to CNB regarding my g...,Credit card or prepaid card,CITY NATIONAL BANK
2,I have been working on completing a short sale...,Mortgage,Ocwen Financial Corporation
3,This complaint is against : KeyBank / Key Corp...,"Payday loan, title loan, or personal loan",KEYCORP
4,This is in reference to closed case XXXX XXXX ...,Debt collection,"Debt Recovery Solutions, LLC"
...,...,...,...
23181,I received a letter dated XX/XX/2021 identifyi...,Mortgage,NATIONSTAR MORTGAGE
23182,Solomon and Solomon Pc is attempting to collec...,Debt collection,"Solomon and Solomon, P.C."
23183,XXXX BAM Trading Services Inc. d.b.a. Binance....,"Money transfer, virtual currency, or money ser...",BAM Management US Holdings Inc.
23184,To Whom it may concern received a certified le...,Credit card or prepaid card,Conduent Incorporated


In [ ]:
pd.set_option('display.max_colwidth', -1)
complaints_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


Complaints  \
0  XXXX, Transunion and XXXX are reporting a Collection Account on my credit report from XXXX. This company NOR did the original creditor sent me anything in writing regarding this debt. Despite my multiple attempts to contact them and asking for the details on what or how I owed them anything.\n\nI disputed the account with all the three of the major credit bureaus and XXXX " verified '' the debt. I filed a complaint with the CFPB ( XXXX-XXXX  ) and the company basically told on themselves when they included two attachments in their response. I was never sent anything in writing to verify the debt, despite asking for it in writing and calling the original creditor repeatedly to obtain it. Instead it was reported on my bureau and to date, XX/XX/2020, I still have NOT received anything in writing. \n\n1. ) The first attachment was the supposed letter they sent to me. It was sent to their own address, not to my forwarding address.\n\n2. ) Second, they sent the rental lease, the lease creation date is recent and not covering the time period, which shows they just generated the lease. \n\nThe letter wasn't mailed to my forwarding address nor to the old rental address with the apartment number included which would have allowed the mail to be forwarded to me successfully. Instead they simply mailed the notice to themselves and then sent the account to collections after returning my payment. A payment I made blindly to prevent damaging my credit report.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.6, random_state=111)

In [ ]:
X_train['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    3322
Mortgage                                                                        1629
Debt collection                                                                 1607
Credit card or prepaid card                                                     1261
Checking or savings account                                                     563 
Vehicle loan or lease                                                           276 
Money transfer, virtual currency, or money service                              255 
Student loan                                                                    193 
Payday loan, title loan, or personal loan                                       168 
Name: Product, dtype: int64

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2) ] 
   #stems = [stemmer.stem(item) for item in tokens]
   return tokens

In [ ]:
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000, use_idf=False, norm=None)
tf_vectors = vectorizer_tf.fit_transform(X_train.Complaints) 

In [ ]:
tf_vectors.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
vectorizer_tf.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['0.00',
 '1.00',
 '10.00',
 '100.00',
 '1000.00',
 '10000.00',
 '1001.',
 '1022',
 '110.00',
 '1100.00',
 '11000.00',
 '120.00',
 '1200.00',
 '12000.00',
 '130.00',
 '1300.00',
 '140.00',
 '1400.00',
 '150.00',
 '1500.00',
 '15000.00',
 '160.00',
 '1600.00',
 '16000.00',
 '1621.',
 '1681',
 '1681168l',
 '1681b',
 '1681c',
 '1681et',
 '1681i',
 '168ls2',
 '1692',
 '1692g',
 '170.00',
 '1700.00',
 '180.00',
 '1800.00',
 '190.00',
 '1900.00',
 '20.00',
 '200.00',
 '2000.00',
 '20000.00',
 '2019',
 '2020',
 '2021',
 '210.00',
 '2100.00',
 '2200.00',
 '2400.00',
 '25.00',
 '250.00',
 '2500.00',
 '2600.00',
 '2700.00',
 '2800.00',
 '290.00',
 '2900.00',
 '30-day',
 '300.00',
 '3000.00',
 '30day',
 '35.00',
 '350.00',
 '3500.00',
 '3700.00',
 '386-pieces',
 '40.00',
 '400.00',
 '4000.00',
 '426-chrc',
 '450.00',
 '4500.00',
 '50.00',
 '500.00',
 '5000.00',
 '600.00',
 '6000.00',
 '605b',
 '61500.',
 '7-10',
 '700.00',
 '800.00',
 'ability',
 'able',
 'absolute',
 'absolutely',
 'absurd',
 'a

In [ ]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=3, learning_method='online', learning_offset=50, n_jobs=-1, random_state=111)

W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

In [ ]:
W1

array([[1.26472346e-01, 4.35106866e-01, 1.09120893e-03, 2.02029865e-01,
        2.17864785e-01, 1.74349294e-02],
       [2.62113998e-03, 2.61234564e-03, 2.60818344e-03, 2.63666948e-03,
        9.86908244e-01, 2.61341716e-03],
       [2.46263498e-03, 2.46349682e-03, 2.45752523e-03, 9.87687240e-01,
        2.46470283e-03, 2.46440057e-03],
       ...,
       [4.31983499e-03, 4.31007799e-03, 4.28845247e-03, 3.85625993e-01,
        5.97160485e-01, 4.29515661e-03],
       [1.59793373e-04, 1.60032533e-04, 1.59900144e-04, 1.59705062e-04,
        1.59610367e-04, 9.99200959e-01],
       [2.76054069e-04, 2.76613655e-04, 9.98618874e-01, 2.75982271e-04,
        2.75812181e-04, 2.76663736e-04]])

In [ ]:
num_words=15

vocab = np.array(vectorizer_tf.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
topics

['loan bank documents property merchant modification information foreclosure case claim sale application attorney state home',
 'credit debt account report reporting information letter collection sent company inquiry request creditor proof provide',
 'claim reporting collection compliance debt claims alleged compliant plaintiff document to-be practices requisite proof complaint',
 'account bank card credit told called number payment said phone time days dispute email money',
 'payment loan mortgage payments told time paid called sent company late account letter email said',
 'information consumer credit reporting report account accounts section dispute agency furnisher fcra file inaccurate identity']

In [ ]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.Complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [ ]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.13,0.44,0.0,0.20,0.22,0.02,1
Doc1,0.00,0.00,0.0,0.00,0.99,0.00,4
Doc2,0.00,0.00,0.0,0.99,0.00,0.00,3
Doc3,0.00,0.00,0.0,0.91,0.08,0.00,3
Doc4,0.00,0.82,0.0,0.00,0.08,0.10,1
...,...,...,...,...,...,...,...
Doc9269,0.00,0.17,0.0,0.50,0.32,0.00,3
Doc9270,0.04,0.51,0.0,0.29,0.12,0.03,1
Doc9271,0.00,0.00,0.0,0.39,0.60,0.00,4
Doc9272,0.00,0.00,0.0,0.00,0.00,1.00,5


In [ ]:
X_train.head()

,Complaints,Product,Company
5222,"On XX/XX/XXXX of XXXX an aggressive and violent tenant, one whom has made multiple physical threats unprovoked and constantly had a man with a known violent criminal record stay with her committed assault and battery against me. The police were called, I had open wounds on my face from where I was clawed and recommended to go to the hospital. I wrote the apartment, XXXX XXXX, a letter and presented proof of the violent assault that took place. As over half a dozen noise complaints had been filed, police have shown up to their residence for unknown reasons, often needing to knock the door down, the threat was forseable. XXXX XXXX violated the terms of our lease agreement and not only failed to provide a quiet and safe environment, but acted negligently in allowing the situation to escalate despite ours and others warnings. Office manager XXXX XXXX agreed. I moved out completely on XX/XX/XXXX and was asked to pay for both XXXX and XXXX rent amounts. As the contract was legally void I offered to not sue for negligence and pay a pro rated amount until XX/XX/XXXX. The keys had been returned to them on XX/XX/XXXX when the incident was reported. I am XXXX and did not leave my apartment unless absolutely necessary. \n\nI have not heard a single thing, received for any mail or email or evening phone call since. As of XX/XX/XXXX there is a {$1200.00} collection on my 3 credit reports from FAIR COLLECTIONS & OUT with the only information provided being the name and the amount listed as derogatory. \n\nWith absolutely nothing to go on I disputed with the three credit report agencies and they directed me to file a report with the FTC. I have done all that I was asked to, sent a letter on XX/XX/XXXX to the debt collector that was generated by the FTC. By this time I had found an address after XXXX the information. No email or phone number were provided to me to this day. \n\nThey have not reached back out to me, but they have responded to the disputes I filed with the reporting agencies regarding their accuracy and requesting information be sent to me at my provided home address. \n\nThey added an account number and date of first delinquency : XXXX ; XX/XX/XXXX. \n\nThe letter is very detailed and in each dispute I made with all three credit reporting agencies I made it clear I have never been notified of any past due amount, sent a bill, or contacted about this. \n\nXXXX XXXX employees have been a nightmare to deal with when I did live there. After my last contact with them I do not want further contact, and will have to sue for Negligence and fraud by early XXXX if no one will actually verify they contacted me, or made any attempt to contact me as if they had I would have responded before this became a legal issue I cant afford with this epidemic living in XXXX.",Debt collection,"FAIR COLLECTIONS & OUTSOURCING, INC."
10312,I took a personal loan out with Ally Bank through XXXX XXXX XXXX in the beginning of XXXX. I received an email on XX/XX/XXXX that my first payment was due on XX/XX/XXXX. I made my first payment on that date. A few days later I received an email that there was an error and my first payment was due on XX/XX/XXXX XXXX I made my payment of the due date that I was told. The end of XXXX I received notification that my payment was late. I called and explained the situation and was told the late fee will be reversed. XXXX I made my normal payment and a few days later I received another notification that my payment was late. When I called customer service I was told my request was denied and i will have to pay the late payment. I explained that is is a financial burden because i am on a fixed income. I work part time but because of Covid my job is shut down. Now the late payment is doubled since I did not make the first late payment. I tried to explain that this was an error on their part so why should i be punished But they dont want to listen. I am at my wits end. If you can help I would great

In [ ]:
WHold = lda.transform(vectorizer_tf.transform(X_hold.Complaints[:5]))

In [ ]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.13,0.44,0.0,0.20,0.22,0.02,1
Doc1,0.00,0.00,0.0,0.00,0.99,0.00,4
Doc2,0.00,0.00,0.0,0.99,0.00,0.00,3
Doc3,0.00,0.00,0.0,0.91,0.08,0.00,3
Doc4,0.00,0.82,0.0,0.00,0.08,0.10,1
...,...,...,...,...,...,...,...
Doc9269,0.00,0.17,0.0,0.50,0.32,0.00,3
Doc9270,0.04,0.51,0.0,0.29,0.12,0.03,1
Doc9271,0.00,0.00,0.0,0.39,0.60,0.00,4
Doc9272,0.00,0.00,0.0,0.00,0.00,1.00,5


In [ ]:
X_hold.head()

Complaints  \
14507  Accounts added and/or created unbeknownst to me Notice to whom it be of concern, It is YOU I hold accountable and accuse of complicity in exercise of malevolent did-compliance. I hereby mark you as likely one that I might one day view in civil discord as the " would be plaintiffs '', yet I declare to you your unjust claim is to be met with deliberate and determined resistance and counteraction. In is in clarity that your Claim of Non-Medical debt Collection is insufficient for collections much less reporting. You MUST DELETE this unjust injurious allegation because it is significantly deficient of the requisite certified reporting compliance ( such as perfect metro 2 format reporting ) and testimonial evidence of the PHYSICALLY VERIFIABLE PROOF OF VALIDATION as related to the full truth, accuracy, completeness, timeliness, ownership, identity of true debtor, dates, balances, audits, calculations, source codes, creditor collection code, 426-chrc P6 statement and its associated trailing fragments, the minimal five portioned personal identifiers, the pay statuses, the required 386-pieces of confirmation to collection, and elsewise standard or mandatory for proper reporting, mentioned or not. \n\n\n\nAlthough I am checking for and addressing missing and or deficient aspects of REPORTING COMPLIANCES and not contesting any debt of compliant nature, I should make you aware that since unlawful reporting transitions collection into an equally not complaint circumstance. Being still yet not validated by document fact in compliance to requisite standards, it is to be announced yet again that legally I have no knowledge of the validity of the alleged claims of delinquency and or derogatory nature, nor of the certifiably compliant matter to either any of its collection attempts and or its reporting despite previous consumer filed composed complaints checking for each. Might it be known, especially shall I elect to take this matter up to a civil court, any debt and or derogatory claim must be pursued ( particularly for collection ) in a very defined and precisely compliant and physically verifiable or certifiable manner as detailed in the requisite obeyed federal and state collection and reporting regulations associated with any of the above noted said claim ( s ) to include but not limited to the FCBA, FCRA, HIPAA PRIVACY RULE, FACTA,  FDCPA and TCPA, etc. ADDITIONALLY, if an entity acts as a collector and also elects to act as a reporting party of consumer credit they must as well adhere to every single one even each any and all of the regulatory reporting requisites and standards of reporting with legal standing in full accordance of laws and accepted reporting standards. To date, the plaintiff has failed to demonstrate any capacity or willingness to validate the alleged debt much less certify the fair, accurate, complete and compliant reporting of the claims, particularly being significantly deficient is any display of certified metro 2 compliance. As such, given the fact of recent breaches of information collection repositories, I am hesitating to readily accept what is presented without testimonial and certified physically verifiable document evidence of claim as being legitimate. I deny nothing, yet I reserve the right to question any unproven claim. I do NOT accept the statement of claim of the plaintiff and I call for the court to reject their claim as untrue, unverified, incomplete, not compliant or otherwise invalid and thereby reject-able and dischargeable which requirement to be dismissed in full. I demand said court resolution today, even here and now in full accords of regulatory statutes, mentioned by me or not. \n\n\n1 ) To date, I have received no knowledge of ever entering into any contractual relationship with the to-be plaintiff ( s ) { should I forward into a civil complaint } or with the entity ( ies ) that allegedly issued against me the defaulted line of credit ( s ) or accusations similar. This